In [2]:
import pandas as pd
df = pd.read_csv("../Data/lstm.csv")
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [3]:
df.category.unique()
# 이진분류

array(['food', 'sports'], dtype=object)

In [6]:
# 데이터에 사용된 중복 없는 전체 단어 갯수를 파악
results = set()
# 소문자로 만들고 쪼갠다
df['paragraph'].str.lower().str.split().apply(results.update)
vocab_size = len(results)
vocab_size

536

In [7]:
# 단어를 숫자로 인코딩
paragraphs = df['paragraph'].to_list()
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [9]:
from tensorflow import keras
keras.utils.set_random_seed(1)

In [10]:
# one hot encoding
encoded_paragraphs = [keras.preprocessing.text.one_hot(paragraph, vocab_size) for paragraph in paragraphs]

In [11]:
# 데이터에서 가장 긴 문장의 단어 갯수 확인
max_length = 0
for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
        max_length = len(row.split(" "))
print(max_length)

91


In [16]:
# 문장마다 단어 갯수가 다르므로 Sequence padding을 넣어서 문장의 길이를 동일하게 만듬
padded_paragraphs_encoding = keras.preprocessing.sequence.pad_sequences(encoded_paragraphs,maxlen=max_length,padding="post")
padded_paragraphs_encoding

array([[ 95, 119, 464, ...,   0,   0,   0],
       [352, 256, 146, ...,   0,   0,   0],
       [ 32, 308, 242, ...,   0,   0,   0],
       ...,
       [470,  25,   8, ...,   0,   0,   0],
       [193, 312,  39, ...,   0,   0,   0],
       [ 72, 476, 312, ...,   0,   0,   0]], dtype=int32)

In [17]:
# 분류형태 (food, sports)로 수치  변경하기 
categories = df['category'].to_list()

def category_encode(category):
    if category == "food":
        return [1,0]
    else:
        return [0,1]

In [18]:
encoded_category = [category_encode(category) for category in categories]
encoded_category

[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [19]:
# 피쳐 확인 
print(encoded_paragraphs[0])

[95, 119, 464, 292, 185, 81, 406, 119, 256, 143, 312, 330, 193, 469, 256, 146, 528, 51, 303, 529, 9, 66, 209, 508, 526, 282, 222, 476, 51, 137, 233, 8, 331, 285, 403, 66, 344, 432, 338, 225, 465, 222, 151, 312, 417, 278, 14, 71, 264, 312, 21, 51, 101]


In [22]:
len(encoded_paragraphs[19])

73

---
# 주제 분류하는 모델 구현학

In [23]:
model = keras.Sequential()

2022-07-22 14:11:19.901136: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
# 문맥 생성단계
# vocab_size = 536
model.add(keras.layers.Embedding(vocab_size,5,input_length=max_length))
# 임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩 출력 => hyper parameter
# 하나의 토큰을 주어, 동사, 목적어, 보어를 알기 위해 5 => 한국어는 필요 없다.
model.add(keras.layers.LSTM(64))

In [27]:
#  분류 단계
# lstm의 반만 
model.add(keras.layers.Dense(32,activation="relu"))
# 출력층
model.add(keras.layers.Dense(2,activation="softmax"))
# 출력하는 게 food, sports이지만 0과 1의 숫자가 아니라 문장형태이므로 다준분류 ,sigmoid


In [28]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [29]:
import numpy as np
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [30]:
# 학습
model.fit(train_X,train_Y,batch_size=10,epochs=50)


Epoch 1/50
2/2 [==============================] - 2s 26ms/step - loss: 0.6938 - accuracy: 0.4000
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6929 - accuracy: 0.5500
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 4/50
2/2 [==============================] - 0s 23ms/step - loss: 0.6924 - accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6919 - accuracy: 0.5000
Epoch 6/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6915 - accuracy: 0.5000
Epoch 7/50
2/2 [==============================] - 0s 21ms/step - loss: 0.6909 - accuracy: 0.5000
Epoch 8/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6901 - accuracy: 0.5000
Epoch 9/50
2/2 [==============================] - 0s 29ms/step - loss: 0.6893 - accuracy: 0.5000
Epoch 10/50
2/2 [==============================] - 0s 26ms/step - loss: 0.6885 - accuracy: 0.5000
Epoch 11/50
2/2 [============